In [1]:
from __future__ import print_function
import os, sys
from collections import defaultdict as dfd


In [2]:
def read_fasta(file):
    ret = []
    for l in file:
        l = l.strip()
        if l.startswith('>'):
            ret.append([l[1:],[]])
        else:
            ret[-1][1].append(l.strip())
    
    return [[x[0], ''.join(x[1])] for x in ret]


In [36]:
def read_fastq(file, size=None):
    ret = []
    line  = 0
    
    while True:
        name = file.readline().strip()
        if not name:
            break
        line +=1
        if size and line> size: 
            break
        seq = file.readline().strip()
        plus = file.readline().strip()
        quality = file.readline().strip()
        ret.append([name,seq,quality])
    return ret


In [44]:
def read_paired_fastq(file1, file2, size=None):
    ret1 = read_fastq(file1, size)
    ret2 = read_fastq(file2, size)
    
    for i in xrange(min(len(ret1), len(ret2))):
        pair = [ret1[i], ret2[i]]
        
        if pair[0][0][-1] != '1':
            raise(Exception('first file does not have first read'+str(pair)))

        if pair[1][0][-1] != '2':
            raise(Exception('second file does not have first read'))
        
        if pair[0][0][:-2] != pair[1][0][:-2]:
            raise(Exception('paired reads name mismatch'))

    return list(zip(ret1,ret2))


In [45]:
fname = '../data/usama/GAML-aureus.fasta'
file = open(fname)
data = read_fasta(file)

In [46]:
fname1 = '../data/usama/original/shortjump_1.fastq'
fname2 = '../data/usama/original/shortjump_2.fastq'
pairs = read_paired_fastq(open(fname1), open(fname2),10**4)

In [35]:
pom = read_fastq(open(fname1))

0 @SRR022865.7/1
10000 @SRR022865.51838/1
20000 @SRR022865.103730/1
30000 @SRR022865.155645/1
40000 @SRR022865.207470/1
50000 @SRR022865.259712/1
60000 @SRR022865.311797/1
70000 @SRR022865.363782/1
80000 @SRR022865.416106/1
90000 @SRR022865.468463/1
100000 @SRR022865.520751/1
110000 @SRR022865.571455/1
120000 @SRR022865.623250/1
130000 @SRR022865.675077/1
140000 @SRR022865.727619/1
150000 @SRR022865.780177/1
160000 @SRR022865.832061/1
170000 @SRR022865.883634/1
180000 @SRR022865.935664/1
190000 @SRR022865.987152/1
200000 @SRR022865.1039677/1
210000 @SRR022865.1091160/1
220000 @SRR022865.1143181/1
230000 @SRR022865.1196172/1
240000 @SRR022865.1247816/1
250000 @SRR022865.1300236/1
260000 @SRR022865.1352129/1
270000 @SRR022865.1404563/1
280000 @SRR022865.1456289/1
290000 @SRR022865.1508246/1
300000 @SRR022865.1559731/1
310000 @SRR022865.1611372/1
320000 @SRR022865.1663405/1
330000 @SRR022865.1714941/1
340000 @SRR022865.1765772/1
350000 @SRR022865.1817772/1
360000 @SRR022865.1868851/1
3700

KeyboardInterrupt: 

In [54]:
def kmeryfie(generator, k):
    ids = {}
    graph = dfd(list)
    for read in generator:
        last = None
        for index in range(len(read)-k+1):
            now = read[index:index+k]
            if now not in ids:
                ids[now] = len(ids)
            if last:
                graph[ids[last]].append(ids[now])
            last = now

    return (ids, graph)

In [63]:
maper, graph = kmeryfie(map(lambda x:x[0][1], pairs),30)

In [66]:
lens = map(lambda x: len(x[1]), graph.items())

In [67]:
sorted(lens)[-10:]

[142, 143, 144, 144, 145, 148, 150, 150, 154, 286]